You should be able to `run all cells` and the notebook should execute to completion. Just the train set itself is 18GB so the download of the files might take a while.

You have to make sure the PATH you give below exists and the notebook should be able to take it from there.

[Official website of the COCO dataset](http://cocodataset.org)

In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
from utils import *

from fastai.conv_learner import *
from fastai.dataset import *

from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
torch.cuda.set_device(0)

In [5]:
print_info()

Last run on: 2018-03-21
PyTorch version: 0.3.1.post2
fastai version: 0.6


In [6]:
PATH = Path('data/coco2017')

In [11]:
%%capture
!wget http://images.cocodataset.org/zips/train2017.zip -P {PATH}
!wget http://images.cocodataset.org/zips/val2017.zip -P {PATH}
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P {PATH}